In [ ]:
#Prepare Data From CSV Recording

##Load File

'Data is loaded from a CSV recording file, accepted through an input prompt. This includes all positional data related to the 6 trackers (HMD, Left Controller, Right Controller, Waist, Left Foot, Right Foot).'

'Data is loaded into a Pandas dataframe. The primary tracking data is then extracted, leaving extraneous data such as booleans for button presses.'

'The extracted columns are then concatenated into a new dataframe, and the columns are renamed for ease of reading.' 

'The new trimmed file is written to a directory (/trim_output), for further manipulation and loading into the model.'


In [ ]:
import numpy as np 
import pandas as pd

recording_path = "../recordings/"
output_path = "../trim_output/"
file_name = input("Input File Name")

#Read in CSV

try:
    dataframe = pd.read_csv(recording_path + file_name + ".csv")
except:
    print("Error Reading File: Check Spelling and Try Again")

# Seperate each tracker to seperate dataframe
HMD = dataframe.iloc[:, 1:4]
controller_1 = dataframe.iloc[:, 7:10]
controller_2 = dataframe.iloc[:, 37:40]
left_foot = dataframe.iloc[:, 67:70]
right_foot = dataframe.iloc[:, 73:76]
waist = dataframe.iloc[:, 79:82]

# Join all trackers together
joined = pd.concat([HMD, controller_1, controller_2,
                   waist, left_foot, right_foot], axis=1)

# set new column headers
joined.columns = [
    "head_x",
    "head_y",
    "head_z",
    "r_controller_x",
    "r_controller_y",
    "r_controller_1_z",
    "l_controller_x",
    "l_controller_y",
    "l_controller_z",
    "waist_x",
    "waist_y",
    "waist_z",
    "r_foot_x",
    "r_foot_y",
    "r_foot_z",
    "l_foot_x",
    "l_foot_y",
    "l_foot_z"
]


# output to new csv
output_file = output_path + file_name + "_trimmed.csv"
joined.to_csv(output_file, index=False)

print(file_name + " output to " + output_path)


In [ ]:
#Data Normalization

## Data Scaling

'The new CSV is loaded into memory, chosen through an input prompt'
'The data is then split between the features (the HMD and controller tracking data), and the labels (the waist and foot trackers).'
'These are loaded into Numpy arrays to peform normaliztion. The output from OpenVR Recorder is upscaled by 100. To correct this the array is divided by 100'

In [4]:
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
#from sklearn.metrics import mean_absolute_error 
from matplotlib import pyplot as plt
#import seaborn as sb
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np


output_path = "../trim_output/"

file_name = input("Input File Name")

#read in formatted CSV
try:
    dataframe = pd.read_csv(output_path + file_name + ".csv")
    print("Dataframe created")
except:
    print("Error Reading File")


#Show shape of the dataframe
dataframe.describe()

x_train_df = dataframe.iloc[:, 0:9]
y_train_df = dataframe.iloc[:, 9:18]
#display(x_train_df)
#display(y_train_df)

#Load data into Numpy array
x_train = np.array(x_train_df)
y_train = np.array(y_train_df)



#Divide values in array by 100
x_train_normalized = np.divide(x_train, 100)
y_train_normalized = np.divide(y_train, 100)

print(x_train_normalized.shape, x_train_normalized)
print(y_train_normalized.shape, y_train_normalized)

#x_train, x_test, y_train, y_test = train_test_split(x_train_normalized, y_train_normalized)








Input File Name arm_raise_1_trimmed


Dataframe created
(905, 9) [[-0.01512766  1.5719162  -0.09888399 ... -0.26088011  0.76942291
  -0.10149074]
 [-0.01512766  1.5719162  -0.09888399 ... -0.26017019  0.76955322
  -0.10172288]
 [-0.01513591  1.5719162  -0.09887324 ... -0.25940933  0.76964912
  -0.10187013]
 ...
 [-0.0277946   1.60442154 -0.08457804 ... -0.19361816  1.17218666
  -0.25626904]
 [-0.0277946   1.60442154 -0.08457804 ... -0.19326099  1.17207565
  -0.2562438 ]
 [-0.0277946   1.60442154 -0.08457804 ... -0.19324625  1.17204033
  -0.25649668]]
(905, 9) [[-0.0044589   0.96929459 -0.08480424 ... -0.20655287  0.09259642
   0.00614011]
 [-0.00440914  0.96923447 -0.08475275 ... -0.20653749  0.09257123
   0.00614011]
 [-0.00486888  0.96952217 -0.08502438 ... -0.20651741  0.09255754
   0.00614011]
 ...
 [-0.01349738  0.97748932 -0.09893991 ... -0.20459211  0.09642521
   0.00729573]
 [-0.01333761  0.97723671 -0.09890824 ... -0.20459616  0.09634519
   0.00729573]
 [-0.01398552  0.97791611 -0.09938544 ... -0.20459616  0.09634

#Model Creation and Training

In [17]:
import tensorflow as tf
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=len(x_train_normalized), output_dim= 16, input_length=9 ),
    tf.keras.layers.Bidirectional(tf.keras.layers.GRU(32)),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()



Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 9, 16)             14480     
                                                                 
 bidirectional_2 (Bidirectio  (None, 64)               9600      
 nal)                                                            
                                                                 
 dense_4 (Dense)             (None, 10)                650       
                                                                 
 dense_5 (Dense)             (None, 1)                 11        
                                                                 
Total params: 24,741
Trainable params: 24,741
Non-trainable params: 0
_________________________________________________________________


In [19]:

# fit the model
model.fit(x_train_normalized, y_train_normalized, epochs=50, verbose=0)
# evaluate the model
loss, accuracy = model.evaluate(x_train_normalized, y_train_normalized, verbose=0)
print('Accuracy: %f' % (accuracy*100))

ValueError: in user code:

    File "C:\Users\Jev\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 1021, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\Jev\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 1010, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\Jev\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 1000, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\Jev\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 860, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "C:\Users\Jev\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 918, in compute_loss
        return self.compiled_loss(
    File "C:\Users\Jev\AppData\Local\